# 階層ベイズモデル
- 事前分布のパラメータに事前分布（階層事前分布）を仮定し、事前分布のパラメータも合わせて推定する
  - データ数に偏りがある場合に、推定の偏りを低減できる
  - サンプルデータのグループの上位の階層を想定したモデルを構築できる

In [ ]:
import sys, os
import numpy as np
import pandas as pd
import scipy.stats as stats
import pymc3 as pm
import math

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# プロットする図を綺麗にする
sns.set()

## データ確認

In [ ]:
data = pd.read_csv('../data/angel_data_for_publish.csv')
tastes = pd.read_csv('../data/choco_tastes.csv')

In [ ]:
data = data.query('taste in [0,1,12]') # 特定の味だけのデータに選別
data.head()

In [ ]:
tastes.head()

In [ ]:
data.groupby(['taste']).count()[['angel']]

In [ ]:
fig = plt.figure(figsize=(16, 4))
ax = fig.subplots(1, 2)

bins = np.linspace(0, 5.0, 30)
grp = data.groupby(['taste'])
for key, value in grp:
    spec = tastes.query('id == {}'.format(key))['spec'].values[0]
    taste_name = tastes.query('id == {}'.format(key))['name_en'].values[0]
    sns.distplot(value["net_weight"], bins=np.linspace(21, 33, 50), hist=True, label=taste_name, ax=ax[0])
    sns.distplot((value["net_weight"] - spec), bins=bins, hist=True, label=taste_name, ax=ax[1])
ax[0].set_xlabel("NetWeight [g]")
ax[1].set_xlabel("(NetWeight - Spec) [g]")
ax[0].legend()
ax[1].legend()
fig.savefig("weight_histogram.png")

In [ ]:
data.groupby(['buyer', 'angel']).count()[['taste']]

## 正味重量を独立にベイズモデルで推定

In [ ]:
taste = data['taste'].values
taste_idx = pd.Categorical(data['taste']).codes
spec_lst = np.array([tastes.query('id == {}'.format(i))['spec'].values[0] for i in set(taste)])
print(spec_lst)
print(set(taste))
print(set(taste_idx))

### モデル設定
- ベイズの式を再度思い出す
    - $p(\theta | x) \propto p(x | \theta)p(\theta)$
    - x : チョコボールの重量データ
    - $\theta$ : 確率分布のパラメータ
    - 尤度$p(x | \theta)$と事前分布$p(\theta)$を設定する必要がある
- 尤度関数
    - 最尤推定と同様に、正規分布を採用
    - $p(x | \theta) = N(\mu, \sigma^2)$
      - なお, $\mu=spec+\alpha$
    - 推定するパラメータは二つ
      - $\alpha$ : 平均のマージン
      - $\sigma^2$ : 分散
- 事前分布
    - 推定するパラメータ毎にそのパラメータの事前分布を設定する必要がある
    - マージン$\alpha$の事前分布には正規分布
      - $p(\alpha) = N(\mu_a, \sigma^2_a)$
    - 分散$\sigma^2$の事前分布には半正規分布
      - $p(\sigma^2) = HalfNormal(sigma^2_b)$

In [ ]:
with pm.Model() as comparing_weight_h:
    # 事前分布
    alpha = pm.Normal('alpha', mu=0, sd=10, shape=len(set(taste_idx)))
    mu = pm.Deterministic('mu', spec_lst[taste_idx]+alpha[taste_idx])
    sds = pm.HalfNormal('sds', sd=10, shape=len(set(taste_idx)))
    # 重量モデル
    weights = pm.Normal('weights', mu=mu, sd=sds[taste_idx], observed=data['net_weight'].values)
    
    trace_wi = pm.sample(5000, chains=3)

In [ ]:
pm.traceplot(trace_wi, varnames=['alpha', 'sds'])

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.subplots(3, 1, sharex=True)

pm.plot_posterior(trace_wi, varnames=['alpha'], 
                  kde_plot=True, alpha_level=0.05, ax=ax)
plt.savefig('posterior_weight_diff_iso.png')

## 正味重量を階層ベイズモデルで推定
- マージンは味毎に変えているわけではなく、森永さんがチョコボールを製造する際の思想で設定しているものと想像
    - 仕様に対してalphaグラム多く入れることにしようと方針を決めていると想像
- そのため、データ数が少ないために、マージンに差が出てしまったのではないかとも考えられる


### モデル
- ベイズの式
    - $p(\theta | x) \propto p(x | \theta)p(\theta)$
    - x : チョコボールの重量データ
    - $\theta$ : 確率分布のパラメータ
    - 尤度$p(x | \theta)$と事前分布$p(\theta)$を設定する必要がある
- 尤度関数
    - 最尤推定と同様に、正規分布を採用
    - $p(x | \theta) = N(\mu, \sigma^2)$
      - なお, $\mu=spec+\alpha$
    - 推定するパラメータは二つ
      - $\alpha$ : 平均のマージン
      - $\sigma^2$ : 分散
- 事前分布
    - 推定するパラメータ毎にそのパラメータの事前分布を設定する必要がある
    - マージン$\alpha$の事前分布には正規分布
      - $p(\alpha) = N(\mu_a, \sigma^2_a)$
    - 分散$\sigma^2$の事前分布には半正規分布
      - $p(\sigma^2) = HalfNormal(sigma^2_b)$
- 階層事前分布
    - 事前分布を統括する分布として階層事前分布を仮定
    - マージン$\alpha$の事前分布のパラメータに階層事前分布を設定する
      - $\mu_a$と$\sigma^2_a$のそれぞれに対して事前分布を設定
      - $\sigma^2_b$に対して事前分布を設定

In [ ]:
with pm.Model() as comparing_weight_h:
    # 階層事前分布
    am = pm.Normal('am', mu=0, sd=10)
    asd = pm.HalfNormal('asd', sd=10)
    ssd = pm.HalfNormal('ssd', sd=10)
    # 事前分布
    alpha = pm.Normal('alpha', mu=am, sd=asd, shape=len(set(taste_idx)))
    mu = pm.Deterministic('mu', spec_lst[taste_idx]+alpha[taste_idx])
    sds = pm.HalfNormal('sds', sd=ssd, shape=len(set(taste_idx)))
    # 重量モデル
    weights = pm.Normal('weights', mu=mu, sd=sds[taste_idx], observed=data['net_weight'].values)
    
    trace_wh = pm.sample(5000, chains=3)

In [ ]:
pm.traceplot(trace_wh, varnames=['am', 'asd', 'ssd', 'alpha', 'sds'])

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.subplots(3, 1, sharex=True)

pm.plot_posterior(trace_wh, varnames=['alpha'], 
                  kde_plot=True, alpha_level=0.05, ax=ax)
plt.savefig('posterior_weight_diff_hi.png')

- ピーナッツとイチゴのマージンには差が無いように見える
- しかし、チョコバナナは設定が異なっているものと統計的に見える

## エンゼルの出現確率を購入者毎に推定

In [ ]:
buyer = data['buyer'].values
buyer_idx = pd.Categorical(data['buyer']).codes
buyer_cat = pd.Categorical(data['buyer']).categories
data['buyer_idx'] = buyer_idx
lst_buyer = list(set(buyer_idx))
print(buyer_cat)
print(set(buyer_idx))

In [ ]:
total_counts = data.groupby(['buyer_idx']).count()['silver'].values
angel_counts = data.query('silver > 0').groupby(['buyer_idx']).count()['silver'].values

print('total_count : {}'.format(total_counts))
print('angel_count : {}'.format(angel_counts))


### 頻度（最尤推定）を計算

In [ ]:
theta_mle = angel_counts/total_counts
print(theta_mle)

fig = plt.figure(figsize=(8, 3))
ax = fig.subplots(1, 1)
cs = ['#FF4500', '#0000FF', '#00F1A1']
for idx in np.arange(0, len(theta_mle)):
    ax.vlines(theta_mle[idx], 0, 1, colors=cs[idx%len(cs)], label=buyer_cat[idx])
ax.set_xlim((0.0, 0.1))
ax.set_xlabel('$\\theta_{MLE}$')
ax.legend()
plt.savefig('buyer_effect_mle.png')

### 階層モデルとして、全体を統括するパラメータがあると仮定
- 真の出現率を決めるパラメータがあるはず
- 購入者毎の運によって↑のパラメータに基づいたパラメータがサンプルされる
- エンゼルの出現は確率$\theta_i$の二項分布
- $\theta$の事前分布はベータ分布
- ベータ分布のパラメータに事前分布を設定

In [ ]:
with pm.Model() as comparing_buyer_m2:
    alpha = pm.HalfCauchy('alpha', beta=10)
    beta = pm.HalfCauchy('beta', beta=10)
    
    theta = pm.Beta('theta', alpha=alpha, beta=beta, shape=len(set(buyer_idx)))
    
    angel = pm.Binomial('angel', n=total_counts[lst_buyer], p=theta[lst_buyer], observed=angel_counts[lst_buyer])
    
    trace_h2 = pm.sample(2000, chains=3, random_seed=100)

In [ ]:
pm.traceplot(trace_h2)

In [ ]:
fig = plt.figure(figsize=(8, 4))
ax = fig.subplots(1, 1)

for i in np.arange(len(buyer_cat)):
    sns.distplot(trace_h2['theta'][:,i], label=buyer_cat[i], ax=ax)
ax.legend()
ax.set_xlabel('angel rate')
ax.set_ylabel('frequent')
plt.savefig('buyer_effect_h2model.png')

In [ ]:
# thetaの事前分布
x = np.linspace(0, 1, 100)
for i in np.random.randint(0, len(trace_h2), size=100):
    pdf = stats.beta(trace_h2['alpha'][i], trace_h2['beta'][i]).pdf(x)
    plt.plot(x, pdf,  'C1', alpha=0.2)

dist = stats.beta(trace_h2['alpha'].mean(), trace_h2['beta'].mean())
pdf = dist.pdf(x)
mode = x[np.argmax(pdf)]
mean = dist.moment(1)
plt.plot(x, pdf, label='mode = {:.2f}\nmean = {:.2f}'.format(mode, mean))

plt.legend(fontsize=14)
plt.xlabel('$\\theta_{prior}$', fontsize=16)
plt.tight_layout()
plt.savefig('buyer_effect_prior_h2model.png', dpi=300, figsize=(5.5, 5.5))
